# Data

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:

from backbone.utils.wfo_utils import run_strategy, run_wfo
from backbone.utils.general_purpose import calculate_units_size, diff_pips

import itertools
import numpy as np
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
from pandas import Timestamp
import numpy as np
import random
from backtesting import Strategy
from backtesting.lib import crossover
import talib as ta

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

random.seed(42)

In [3]:
INITIAL_CASH = 10_000
MARGIN = 1/30
COMMISSION = 7e-4

timezone = pytz.timezone("Etc/UTC")
date_from_get_data = datetime(2021, 10, 1, tzinfo=timezone)
date_to_get_data = datetime(2024, 9, 1, tzinfo=timezone)

#
limited_testing_start_date = Timestamp('2023-09-01 00:00:00', tz='UTC')
limited_testing_end_date = Timestamp('2024-09-01 00:00:00', tz='UTC')

In [4]:
groups = [
    'Forex',
    'Cash CFD',
    'Cash II CFD',
    'Metals CFD',
    'Crypto CFD',
    'Exotics',
    'Equities I CFD',
    'Equities II CFD',
    'Agriculture',
    'Commodities'
]

In [5]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

symbols = mt5.symbols_get()

tickers = [symbol.path.split('\\')[1] for symbol in symbols if (
    ('Agriculture' in symbol.path)
    or ('Cash CFD' in symbol.path)
    or ('Cash II CFD' in symbol.path)
    or ('Crypto CFD' in symbol.path)
    or ('Equities I CFD' in symbol.path)
    or ('Equities II CFD' in symbol.path)
    or ('Agriculture' in symbol.path)
    or ('Equities II CFD' in symbol.path)
    or ('Commodities' in symbol.path)
    # or ('Forex' in symbol.path)
    )
]

print(tickers)

intervals = [
    mt5.TIMEFRAME_H6,
    mt5.TIMEFRAME_H4,
    mt5.TIMEFRAME_H3,
    mt5.TIMEFRAME_H2,
    mt5.TIMEFRAME_H1,
    # mt5.TIMEFRAME_M15,
]

symbols = get_data(tickers, intervals, date_from_get_data, date_to_get_data)

['GER40.cash', 'UK100.cash', 'US100.cash', 'US30.cash', 'US500.cash', 'AUS200.cash', 'EU50.cash', 'FRA40.cash', 'HK50.cash', 'JP225.cash', 'N25.cash', 'SPN35.cash', 'US2000.cash', 'UKOIL.cash', 'USOIL.cash', 'ADAUSD', 'BTCUSD', 'DASHUSD', 'DOGEUSD', 'DOTUSD', 'ETHUSD', 'LTCUSD', 'NEOUSD', 'XMRUSD', 'XRPUSD', 'AAPL', 'AMZN', 'BABA', 'BAC', 'GOOG', 'MSFT', 'NFLX', 'NVDA', 'PFE', 'RACE', 'T', 'TSLA', 'V', 'WMT', 'ZM', 'AIRF', 'ALVG', 'BAYGn', 'DBKGn', 'IBE', 'LVMH', 'VOWG_p', 'META', 'COCOA.c', 'COFFEE.c', 'CORN.c', 'SOYBEAN.c', 'WHEAT.c', 'NATGAS.cash']
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
GER40.cash
GER40.cash
GER40.cash
GER40.cash
GER40.cash
UK100.cash
UK100.cash
UK100.cash
UK100.cash
UK100.cash
US100.cash
US100.cash
US100.cash
US100.cash
US100.cash
US30.cash
US30.cash
US30.cash
US30.cash
US30.cash
US500.cash
US500.cash
US500.cash
US500.cash
US500.cash
AUS200.cash
AUS200.cash
AUS200.cash
AUS200.cash
AUS200.cash
EU50.cash
EU50.cash
EU50.cas

In [6]:
max_start_date = None
intervals_start_dates = {}

tickers = np.unique(list(symbols.keys())).tolist()

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date


intervals_start_dates

print(intervals_start_dates)
print(limited_testing_start_date)
print(limited_testing_end_date)

{16390: Timestamp('2024-10-23 00:00:00+0000', tz='UTC'), 16388: Timestamp('2024-10-23 00:00:00+0000', tz='UTC'), 16387: Timestamp('2024-10-23 00:00:00+0000', tz='UTC'), 16386: Timestamp('2024-10-23 00:00:00+0000', tz='UTC'), 16385: Timestamp('2024-10-23 01:00:00+0000', tz='UTC')}
2023-09-01 00:00:00+00:00
2024-09-01 00:00:00+00:00


# Estrategia

In [7]:
from backbone.bbands_cross_strategy import BbandsCross

STRATEGY = BbandsCross

# Analisis preliminar

In [8]:
strategies = [
    STRATEGY
]

experiments = list(itertools.product(
    tickers, intervals, strategies
))

stats_per_symbol = {}

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:
    try:
        print(ticker, interval)
        
        if ticker not in stats_per_symbol.keys():
            stats_per_symbol[ticker] = {}
        
        prices = symbols[ticker][interval].loc[limited_testing_start_date:limited_testing_end_date]

        df_stats, stats = run_strategy(
            strategy=strategy,
            ticker=ticker,
            interval=interval,
            commission=COMMISSION, 
            prices=prices, 
            initial_cash=INITIAL_CASH, 
            margin=MARGIN
        )

        performance = pd.concat([performance, df_stats])
        stats_per_symbol[ticker][interval] = stats

    except Exception as e:
        print(f'hubo un problema con {ticker} {interval}: {e}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


AAPL 16390
AAPL 16388
AAPL 16387
AAPL 16386
AAPL 16385
ADAUSD 16390
ADAUSD 16388
ADAUSD 16387
ADAUSD 16386
ADAUSD 16385
AIRF 16390
AIRF 16388
AIRF 16387
AIRF 16386
AIRF 16385
ALVG 16390
ALVG 16388
ALVG 16387
ALVG 16386
ALVG 16385
AMZN 16390
AMZN 16388
AMZN 16387
AMZN 16386
AMZN 16385
AUS200.cash 16390
AUS200.cash 16388
AUS200.cash 16387
AUS200.cash 16386
AUS200.cash 16385
BABA 16390
BABA 16388
BABA 16387
BABA 16386
BABA 16385
BAC 16390
BAC 16388
BAC 16387
BAC 16386
BAC 16385
BAYGn 16390
BAYGn 16388
BAYGn 16387
BAYGn 16386
BAYGn 16385
BTCUSD 16390
BTCUSD 16388
BTCUSD 16387
BTCUSD 16386
BTCUSD 16385
COCOA.c 16390
COCOA.c 16388
COCOA.c 16387


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:90: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:90: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:90: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(


COCOA.c 16386
COCOA.c 16385


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:90: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:90: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(


COFFEE.c 16390
COFFEE.c 16388
COFFEE.c 16387
COFFEE.c 16386
COFFEE.c 16385
CORN.c 16390
CORN.c 16388
CORN.c 16387
CORN.c 16386
CORN.c 16385
DASHUSD 16390
DASHUSD 16388
DASHUSD 16387
DASHUSD 16386
DASHUSD 16385
DBKGn 16390
DBKGn 16388
DBKGn 16387
DBKGn 16386
DBKGn 16385
DOGEUSD 16390
DOGEUSD 16388
DOGEUSD 16387
DOGEUSD 16386
DOGEUSD 16385
DOTUSD 16390
DOTUSD 16388
DOTUSD 16387
DOTUSD 16386
DOTUSD 16385
ETHUSD 16390
ETHUSD 16388
ETHUSD 16387
ETHUSD 16386
ETHUSD 16385
EU50.cash 16390
EU50.cash 16388
EU50.cash 16387
EU50.cash 16386
EU50.cash 16385
FRA40.cash 16390
FRA40.cash 16388
FRA40.cash 16387
FRA40.cash 16386
FRA40.cash 16385
GER40.cash 16390
GER40.cash 16388
GER40.cash 16387
GER40.cash 16386
GER40.cash 16385
GOOG 16390
GOOG 16388
GOOG 16387
GOOG 16386
GOOG 16385
HK50.cash 16390
HK50.cash 16388
HK50.cash 16387
HK50.cash 16386
HK50.cash 16385
IBE 16390
IBE 16388
IBE 16387
IBE 16386
IBE 16385
JP225.cash 16390
JP225.cash 16388
JP225.cash 16387
JP225.cash 16386
JP225.cash 16385
LTCUSD 163

In [11]:

filter_performance = performance[
    (performance['return/dd']>2)
].sort_values(by=['return/dd'], ascending=[False]).drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BbandsCross,BAC,16388,0.695768,1.186245,10118.624453,0.010257,69 days 00:00:00,100.000000,1.642068,2,3.462525,1.400000,10118.624453,364 days 04:00:00,115.647907,1.289991
0,BbandsCross,TSLA,16390,0.826784,2.005959,10200.595913,0.206230,5 days 06:00:00,100.000000,2.195434,3,9.238317,6.972112,10200.595913,364 days 06:00:00,9.726825,2.305407
0,BbandsCross,BABA,16390,0.788004,0.778730,10077.872978,0.154085,179 days 06:00:00,100.000000,1.452971,2,3.171137,1.992032,10077.872978,364 days 06:00:00,5.053882,0.741299
0,BbandsCross,AAPL,16388,0.731050,1.314096,10131.409578,0.326612,103 days 20:00:00,100.000000,1.526564,3,2.832318,4.000000,10131.409578,364 days 04:00:00,4.023416,1.373215
0,BbandsCross,WMT,16388,0.700070,2.755925,10275.592526,0.689847,15 days 04:00:00,100.000000,1.494238,4,2.738994,8.000000,10275.592526,364 days 04:00:00,3.994981,2.624788
0,BbandsCross,US100.cash,16388,0.921334,4.606671,10460.667125,1.190950,90 days 04:00:00,78.571429,2.041155,14,0.655502,6.809339,10460.667125,364 days 20:00:00,3.868063,5.693920
0,BbandsCross,XMRUSD,16387,0.957555,4.797422,10479.742179,1.347169,52 days 09:00:00,83.333333,1.852057,30,1.296213,16.231278,10479.742179,366 days 00:00:00,3.561113,7.018789
0,BbandsCross,RACE,16390,0.688046,1.029933,10102.993268,0.324216,17 days 00:00:00,100.000000,1.346448,2,3.159250,4.581673,10102.993268,364 days 06:00:00,3.176689,0.854465
0,BbandsCross,ZM,16387,0.834274,2.539376,10253.937609,0.800040,109 days 00:00:00,100.000000,1.775152,6,2.224576,11.717710,10253.937609,364 days 06:00:00,3.174062,2.745160
0,BbandsCross,AIRF,16386,0.830265,4.015003,10401.500292,1.289996,70 days 18:00:00,77.777778,1.408930,9,1.902977,7.299843,10401.500292,364 days 08:00:00,3.112416,4.037076


In [16]:
filter_performance = performance[performance['ticker'].isin(portfolio)].sort_values(by=['ticker', 'interval'], ascending=[False, True])[
    [
        'strategy',
        'ticker',
        'interval',
        'stability_ratio',
        'trades',
        'return',
        'drawdown',
        'return/dd',
        'custom_metric'
        ]
]#.drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance.head(50))

pd.DataFrame(filter_performance.groupby(['ticker']).agg(
    {
        'return/dd':['min', 'max', 'mean','std', 'median'],
    }
).sort_values(by=('return/dd', 'mean')))

,strategy,ticker,interval,stability_ratio,trades,return,drawdown,return/dd,custom_metric
0,BbandsCross,ZM,16385,0.100856,13,0.107411,2.279987,0.047110,0.086422
0,BbandsCross,ZM,16386,0.826335,7,-3.756022,3.900815,-0.962881,-1.593700
0,BbandsCross,ZM,16387,0.834274,6,2.539376,0.800040,3.174062,2.745160
0,BbandsCross,ZM,16388,0.577053,2,0.993524,1.017127,0.976795,0.541115
0,BbandsCross,ZM,16390,0.566929,2,1.213193,0.570894,2.125077,0.848453
0,BbandsCross,XMRUSD,16385,0.734493,71,1.877874,3.151985,0.595775,1.934265
0,BbandsCross,XMRUSD,16386,0.612215,48,1.660137,2.422349,0.685342,1.887871
0,BbandsCross,XMRUSD,16387,0.957555,30,4.797422,1.347169,3.561113,7.018789
0,BbandsCross,XMRUSD,16388,0.712467,17,-1.501974,3.407288,-0.440812,-0.985019
0,BbandsCross,XMRUSD,16390,0.077474,12,0.664873,1.995030,0.333265,0.569399


return/dd                                            
                  min         max       mean        std    median
ticker                                                           
NFLX        -0.985611    2.041094  -0.162179   1.263158 -0.634694
IBE         -0.599448    2.150904  -0.020425   1.214224 -0.549243
COFFEE.c    -0.827831    2.348546   0.035282   1.366274 -0.770281
SOYBEAN.c   -0.914274    2.094900   0.249046   1.228208 -0.313453
US2000.cash -0.747431    2.131060   0.265537   1.258948 -0.484152
AMZN        -0.812431    2.085716   0.304238   1.410630 -0.643152
AIRF        -0.530674    3.112416   0.478273   1.529660 -0.333516
XMRUSD      -0.440812    3.561113   0.946936   1.527097  0.595775
ALVG        -0.460126    3.026385   0.994289   1.388370  1.179630
RACE        -1.000000    3.176689   1.048961   1.563426  0.811611
ZM          -0.962881    3.174062   1.072033   1.637669  0.976795
AAPL        -0.936810    4.023416   1.194299   2.003805  0.379887
US100.cash  -0.499788    3.868063   1.268882   1.770471  1.449854
BABA        -0.301846    5.053882   1.544219   2.241700  0.727647
WMT         -0.228267    3.994981   1.836329   1.917002  0.994279
PFE          0.111366    2.739122   2.081431   1.120039  2.638431
TSLA        -0.997799    9.726825   2.480321   4.269811  1.972909
BAC         -0.878456  115.647907  24.217617  51.169849  0.795623

In [17]:
filter_performance = performance[
    ((performance['ticker'] == 'US100.cash')
    | (performance['ticker'] == 'BABA')
    | (performance['ticker'] == 'WMT')
    | (performance['ticker'] == 'PFE')
    | (performance['ticker'] == 'TSLA')
    | (performance['ticker'] == 'BAC'))
    & (performance['interval'] > 16385)
].sort_values(by=['ticker'], ascending=[False])

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

filter_performance

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BbandsCross,WMT,16386,0.007627,-0.539597,9946.040306,2.363886,263 days 04:00:00,60.000000,0.000000,5,0.067666,14.814815,9946.040306,364 days 06:00:00,-0.228267,-0.287414
0,BbandsCross,WMT,16387,0.126504,1.197086,10119.708629,1.805567,110 days 00:00:00,71.428571,0.510660,7,0.789770,10.785619,10119.708629,364 days 06:00:00,0.662997,0.887261
0,BbandsCross,WMT,16388,0.700070,2.755925,10275.592526,0.689847,15 days 04:00:00,100.000000,1.494238,4,2.738994,8.000000,10275.592526,364 days 04:00:00,3.994981,2.624788
0,BbandsCross,WMT,16390,0.459981,1.944788,10194.478771,0.517553,15 days 06:00:00,100.000000,1.097454,3,2.825796,5.378486,10194.478771,364 days 06:00:00,3.757656,1.776575
0,BbandsCross,US100.cash,16386,0.010006,-0.923942,9907.605798,3.499799,188 days 08:00:00,65.517241,0.000000,29,-0.075180,11.688312,9907.605798,364 days 22:00:00,-0.263999,-0.698367
0,BbandsCross,US100.cash,16387,0.212608,-1.030889,9896.911132,2.062650,181 days 21:00:00,70.588235,0.000000,17,-0.097890,8.706226,9896.911132,364 days 21:00:00,-0.499788,-0.972900
0,BbandsCross,US100.cash,16388,0.921334,4.606671,10460.667125,1.190950,90 days 04:00:00,78.571429,2.041155,14,0.655502,6.809339,10460.667125,364 days 20:00:00,3.868063,5.693920
0,BbandsCross,US100.cash,16390,0.822517,2.728096,10272.809616,1.523839,86 days 18:00:00,90.909091,1.429523,11,0.644696,10.465116,10272.809616,364 days 18:00:00,1.790279,2.686013
0,BbandsCross,TSLA,16388,0.647586,5.671108,10567.110845,2.806593,51 days 04:00:00,100.000000,1.544039,4,15.021218,19.600000,10567.110845,364 days 04:00:00,2.020638,2.397760
0,BbandsCross,TSLA,16386,0.461240,5.396310,10539.630987,2.735204,156 days 20:00:00,71.428571,1.150158,7,2.587934,22.700000,10539.630987,364 days 06:00:00,1.972909,3.004203


In [ ]:
for ticker, interval in zip(portfolio, intervals):
    
    prices = symbols[ticker][interval].loc[limited_testing_start_date:limited_testing_end_date]

    
    df_stats = run_strategy(
        strategy=STRATEGY,
        ticker=ticker,
        interval=interval,
        commission=COMMISSION, 
        prices=prices, 
        initial_cash=INITIAL_CASH, 
        margin=MARGIN,
        plot=True
    )
    break


# WFO

In [ ]:
lookback_bars_per_interval = {
    16388: 1200,
    16386: 1800,
}

In [ ]:
from backbone.utils.wfo_utils import optimization_function

validation_bars = 200
warmup_bars = 200

all_wfo_performances = pd.DataFrame()
all_opt_params = {}

params = {
    'bband_std' : list(np.arange(1, 3, 0.2)),
    'atr_multiplier':list(np.arange(1, 5, 0.3).round(2)),
}

for index, row in filter_performance.iterrows():
    
    try:
        ticker = row.ticker
        interval = row.interval
        
        lookback_bars = lookback_bars_per_interval[interval]
        
        print(ticker, interval)
        
        prices = symbols[ticker][interval]

        wfo_stats, df_stats, opt_params = run_wfo(
            strategy=STRATEGY,
            ticker=ticker,
            interval=interval,
            prices=prices,
            initial_cash=INITIAL_CASH,
            commission=COMMISSION,
            margin=MARGIN,
            optim_func=optimization_function,
            params=params,
            lookback_bars=lookback_bars,
            warmup_bars=warmup_bars,
            validation_bars=validation_bars,
            plot=False
        )
                
        if ticker not in all_opt_params.keys():
            all_opt_params[ticker] = {}

        all_opt_params[ticker][interval] = opt_params

        all_wfo_performances = pd.concat([all_wfo_performances, df_stats])
    
    except Exception as e:
        print(f'No se pudo ejecutar para el ticker {ticker}: {e}')
    
all_wfo_performances['return/dd'] = all_wfo_performances['return'] / -all_wfo_performances['drawdown']
all_wfo_performances['drawdown'] = -all_wfo_performances['drawdown']
all_wfo_performances['custom_metric'] = (all_wfo_performances['return'] / (1 + all_wfo_performances.drawdown)) * np.log(1 + all_wfo_performances.trades)

all_wfo_performances.drawdown_duration = pd.to_timedelta(all_wfo_performances.drawdown_duration)
all_wfo_performances.drawdown_duration = all_wfo_performances.drawdown_duration.dt.days

In [ ]:
# for k, v in params.items():
#     v['risk'] = 10

# params

In [ ]:
filtered_wfo_performance = performance[
    # (performance['stability_ratio'] > 0.65)
    (performance['return/dd'] > 2)
].sort_values(by='return/dd', ascending=False)

filtered_wfo_performance

In [ ]:

performance = pd.DataFrame()

wfo_stats_per_symbol = {}

for index, row in filter_performance.iterrows():
    # try:
    ticker = row.ticker
    interval = row.interval
    print(ticker, interval)
    
    prices = symbols[ticker][interval]
    params = all_opt_params[ticker][interval]
    
    if ticker not in wfo_stats_per_symbol.keys():
        wfo_stats_per_symbol[ticker] = {}

    df_stats, wfo_stats = run_strategy(
        strategy=strategy,
        ticker=ticker,
        interval=interval,
        commission=COMMISSION, 
        prices=prices, 
        initial_cash=INITIAL_CASH, 
        margin=MARGIN, 
        opt_params=params, 
        plot=True
    )

    wfo_stats_per_symbol[ticker][interval] = wfo_stats

    performance = pd.concat([performance, df_stats])

    # except Exception as e:
    #     print(f'hubo un problema con {ticker} {interval}: {e}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


In [ ]:
filtered_wfo_performance.ticker.values

# Montecarlo

In [ ]:
from backbone.utils.montecarlo_utils import montecarlo_statistics_simulation

# Crear una lista para almacenar los resultados de cada ticker
data_drawdown = []
data_return = []
montecarlo_simulations = {}

all_drawdowns = pd.DataFrame()
all_returns = pd.DataFrame()

for index, row in filtered_wfo_performance.iterrows():
    ticker = row.ticker
    interval = row.interval
    
    print(f"Procesando ticker: {ticker}")
    trades_history = stats_per_symbol[ticker][interval]._trades
    eq_curve = stats_per_symbol[ticker][interval]._equity_curve
    
    # Simulación de Montecarlo para cada ticker (datos agregados)
    mc, synthetic_drawdown_curve, synthetic_return_curve = montecarlo_statistics_simulation(
        equity_curve=eq_curve,
        trade_history=trades_history, 
        n_simulations=100_000, 
        initial_equity=INITIAL_CASH, 
        threshold_ruin=0.8, 
        return_raw_curves=True,
        percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
    )
    
    montecarlo_simulations[ticker] = mc
    
    synthetic_drawdown_curve['ticker'] = ticker
    synthetic_return_curve['ticker'] = ticker
    
    all_drawdowns = pd.concat([all_drawdowns, synthetic_drawdown_curve])
    all_returns = pd.concat([all_returns, synthetic_return_curve])

In [ ]:
dd_df = pd.DataFrame()
returns_df = pd.DataFrame()

for ticker, mc in montecarlo_simulations.items():
    mc = mc.rename(
        columns={
            'Drawdown (%)': f'drawdown_{ticker}',
            'Final Return (%)': f'return_{ticker}',
        }
    )

    if dd_df.empty:
        dd_df = mc[[f'drawdown_{ticker}']]
    
    else:
        dd_df = pd.merge(
            dd_df,
            mc[[f'drawdown_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
    if returns_df.empty:
        returns_df = mc[[f'return_{ticker}']]
    
    else:
        returns_df = pd.merge(
            returns_df,
            mc[[f'return_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
display(dd_df)
display(returns_df)


In [ ]:
# Configurar el gráfico con matplotlib y seaborn
plt.figure(figsize=(25, 18))
sns.boxplot(data=all_drawdowns, x="ticker", y="Drawdown (%)")
plt.title("Comparación de Drawdown (%) entre Tickers")

y_max = all_drawdowns["Drawdown (%)"].max()  # Valor máximo en el eje Y
y_min = all_drawdowns["Drawdown (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 2  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()


In [ ]:
# Generar el gráfico
plt.figure(figsize=(12, 20))
sns.boxplot(data=all_returns, x="ticker", y="Final Return (%)")
plt.title("Comparación de Retorno (%) entre Tickers")

# Configurar ticks mayores con más números
y_max = all_returns["Final Return (%)"].max()  # Valor máximo en el eje Y
y_min = all_returns["Final Return (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 10  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()
